# Data cleaning
We want the class with labels as numerical value and the body with clean text.

This will:
* Remove duplicates
* Remove NaN entires


In [1]:
import pandas as pd
import sys
sys.path.append("../../scripts_shared/")
from preprocess_text import preprocess_text


In [2]:
priority = pd.read_csv("csv/full_dataset_priority_high_or_not_high.csv" , index_col=0)
priority

,id,type,created_at,repo,repo_url,action,title,labels,body,class
0,2.125950e+10,IssuesEvent,2022-04-13 01:29:38,federicazoe/gradetools,https://api.github.com/repos/federicazoe/grade...,opened,Comprehensive vignette,high priority,- [ ] Create new vignette on project team grad...,high_priority
1,2.125952e+10,IssuesEvent,2022-04-13 01:31:43,darrylsyms/fretwise-app,https://api.github.com/repos/darrylsyms/fretwi...,closed,App crash,bug requires testing high priority,"App crashes when Darryl visits ""MessagesScreen...",high_priority
2,2.125953e+10,IssuesEvent,2022-04-13 01:31:57,ArctosDB/arctos,https://api.github.com/repos/ArctosDB/arctos,closed,Need a graceful error message for logging in,Priority-Normal (Not urgent) Enhancement Displ...,If you click on a link from your email without...,high_priority
3,2.125953e+10,IssuesEvent,2022-04-13 01:31:58,ArctosDB/arctos,https://api.github.com/repos/ArctosDB/arctos,closed,rebuild startup stuff,Priority-Critical (Arctos is broken),"missing, onRequestStart, rolecheck don't alway...",high_priority
4,2.125953e+10,IssuesEvent,2022-04-13 01:31:59,ArctosDB/arctos,https://api.github.com/repos/ArctosDB/arctos,closed,Error pre-creating locality using geography sp...,Priority-Critical (Arctos is broken) Function-...,**Describe the bug**\r\nI noticed I now have t...,high_priority
...,...,...,...,...,...,...,...,...,...,...
879065,1.475140e+10,IssuesEvent,2021-01-08 05:00:49,nvim-treesitter/nvim-treesitter,https://api.github.com/repos/nvim-treesitter/n...,opened,highlighting works as unexpected after attach ...,bug,**Describe the bug**\r\nsee reproduce.\r\n\r\n...,not_high_priority
879066,1.475140e+10,IssuesEvent,2021-01-08 05:00:51,supabase/ui,https://api.github.com/repos/supabase/ui,closed,Input component has an extra grid space y,bug,# Bug report\r\n\r\nI noticed that the Input c...,not_high_priority
879067,1.475140e+10,IssuesEvent,2021-01-08 05:00:51,supabase/ui,https://api.github.com/repos/supabase/ui,closed,Button to support Ref forwarding,enhancement,# Feature request\r\n\r\n## Is your feature re...,not_high_priority
879068,1.475140e+10,IssuesEvent,2021-01-08 05:01:02,tristanjohnson849/card-demo,https://api.github.com/repos/tristanjohnson849...,closed,We need to merge type 1 and type 2,good first issue,This is purely for practice and fun,not_high_priority


In [3]:
#Number of issues per class
priority["class"].value_counts()

class
not_high_priority    797948
high_priority         80831
Name: count, dtype: int64

In [4]:
# Split the dataframe 
high_priority = priority.loc[priority['class'] == 'high_priority'].copy()
not_high_priority = priority.loc[priority['class'] == 'not_high_priority'].copy()

In [5]:
high_priority.info()

<class 'pandas.core.frame.DataFrame'>
Index: 80831 entries, 0 to 80840
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          80831 non-null  float64
 1   type        80831 non-null  object 
 2   created_at  80831 non-null  object 
 3   repo        80831 non-null  object 
 4   repo_url    80831 non-null  object 
 5   action      80831 non-null  object 
 6   title       80831 non-null  object 
 7   labels      80831 non-null  object 
 8   body        71599 non-null  object 
 9   class       80831 non-null  object 
dtypes: float64(1), object(9)
memory usage: 6.8+ MB


In [6]:
#Give each priority a label by number.
# 'Label encoding'. Makes is easier for machine learning models to work with categorical data.
high_priority["label"] = 0
high_priority.head()

,id,type,created_at,repo,repo_url,action,title,labels,body,class,label
0,2.125950e+10,IssuesEvent,2022-04-13 01:29:38,federicazoe/gradetools,https://api.github.com/repos/federicazoe/grade...,opened,Comprehensive vignette,high priority,- [ ] Create new vignette on project team grad...,high_priority,0
1,2.125952e+10,IssuesEvent,2022-04-13 01:31:43,darrylsyms/fretwise-app,https://api.github.com/repos/darrylsyms/fretwi...,closed,App crash,bug requires testing high priority,"App crashes when Darryl visits ""MessagesScreen...",high_priority,0
2,2.125953e+10,IssuesEvent,2022-04-13 01:31:57,ArctosDB/arctos,https://api.github.com/repos/ArctosDB/arctos,closed,Need a graceful error message for logging in,Priority-Normal (Not urgent) Enhancement Displ...,If you click on a link from your email without...,high_priority,0
3,2.125953e+10,IssuesEvent,2022-04-13 01:31:58,ArctosDB/arctos,https://api.github.com/repos/ArctosDB/arctos,closed,rebuild startup stuff,Priority-Critical (Arctos is broken),"missing, onRequestStart, rolecheck don't alway...",high_priority,0
4,2.125953e+10,IssuesEvent,2022-04-13 01:31:59,ArctosDB/arctos,https://api.github.com/repos/ArctosDB/arctos,closed,Error pre-creating locality using geography sp...,Priority-Critical (Arctos is broken) Function-...,**Describe the bug**\r\nI noticed I now have t...,high_priority,0


In [7]:
not_high_priority["label"] = 1
not_high_priority.head()

,id,type,created_at,repo,repo_url,action,title,labels,body,class,label
80841,1.675120e+10,IssuesEvent,2021-06-12 00:00:03,SFDO-Tooling/CumulusCI,https://api.github.com/repos/SFDO-Tooling/Cumu...,closed,Add installation info section to release notes,enhancement,Currently we generate release notes and then m...,not_high_priority,1
80842,1.675120e+10,IssuesEvent,2021-06-12 00:00:15,atatkin/milos-uptime,https://api.github.com/repos/atatkin/milos-uptime,closed,🛑 Minted HTTPS is down,minted-https status,In [`03a8446`](https://github.com/atatkin/milo...,not_high_priority,1
80843,1.675120e+10,IssuesEvent,2021-06-12 00:00:21,longturn/freeciv21,https://api.github.com/repos/longturn/freeciv21,closed,Client crashes when trying to load the aviatio...,bug,"More specifically, [this version](https://raw....",not_high_priority,1
80844,1.675120e+10,IssuesEvent,2021-06-12 00:00:26,microsoft/vscode,https://api.github.com/repos/microsoft/vscode,closed,Native Notebooks - Changing kernel causes mark...,bug candidate notebook,- VS Code Version: 1.57.0 & 1.57.0-insider\r\n...,not_high_priority,1
80845,1.675120e+10,IssuesEvent,2021-06-12 00:00:27,hashicorp/terraform-provider-kubernetes,https://api.github.com/repos/hashicorp/terrafo...,closed,Feature Request: Generate Resources from Kuber...,acknowledged enhancement stale,"At this point in time, the terraform provider ...",not_high_priority,1


In [8]:
label_counts = priority["class"].value_counts()
print(label_counts)
not_hp = label_counts["not_high_priority"]
print(not_hp)
hp = label_counts["high_priority"]
not_hp

class
not_high_priority    797948
high_priority         80831
Name: count, dtype: int64
797948


797948

In [9]:
not_high_priority = not_high_priority.sample(frac=hp/not_hp)

In [10]:
priority = pd.concat([high_priority, not_high_priority] , ignore_index = True)
priority.tail()

,id,type,created_at,repo,repo_url,action,title,labels,body,class,label
161657,1.472557e+10,IssuesEvent,2021-01-06 05:04:49,breindy/autoCraigslistScraper,https://api.github.com/repos/breindy/autoCraig...,closed,Create Web Scraper Endpoint,Backend Web Scraper,with AlertListings Collections passed in to q...,not_high_priority,1
161658,1.666113e+10,IssuesEvent,2021-06-06 10:45:19,XanderTerbl1/peak-planner,https://api.github.com/repos/XanderTerbl1/peak...,closed,DateTime not updated if passed through custom ...,bug,For some reason the DateTime (in Planner) is n...,not_high_priority,1
161659,1.469324e+10,IssuesEvent,2021-01-03 07:45:57,apache/incubator-doris,https://api.github.com/repos/apache/incubator-...,closed,Some errors in the document need to be rectified,kind/documentation,"Hi, all.\r\nI found some mistakes in our docum...",not_high_priority,1
161660,1.727417e+10,IssuesEvent,2021-07-23 02:15:39,operate-first/alerts,https://api.github.com/repos/operate-first/alerts,closed,CephMonHighNumberOfLeaderChanges,alert:boom: environment/moc/zero resolved,<!-- ID: 0x7b7d2f7b7d2f7b6578706f727465645f6e6...,not_high_priority,1
161661,1.704859e+10,IssuesEvent,2021-07-06 05:33:06,near/near-indexer-for-explorer,https://api.github.com/repos/near/near-indexer...,closed,Track testnet/mainnet releases and deploy inde...,SRE,"@chefsale I believe you already work on this, ...",not_high_priority,1


In [11]:
# Drop duplicates by the content of the body
all_priority = priority.drop_duplicates(subset=['body'], keep='last')

In [12]:
all_priority.dropna(inplace=True)

/tmp/ipykernel_321717/2768040423.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_priority.dropna(inplace=True)


In [13]:
all_priority

,id,type,created_at,repo,repo_url,action,title,labels,body,class,label
0,2.125950e+10,IssuesEvent,2022-04-13 01:29:38,federicazoe/gradetools,https://api.github.com/repos/federicazoe/grade...,opened,Comprehensive vignette,high priority,- [ ] Create new vignette on project team grad...,high_priority,0
1,2.125952e+10,IssuesEvent,2022-04-13 01:31:43,darrylsyms/fretwise-app,https://api.github.com/repos/darrylsyms/fretwi...,closed,App crash,bug requires testing high priority,"App crashes when Darryl visits ""MessagesScreen...",high_priority,0
3,2.125953e+10,IssuesEvent,2022-04-13 01:31:58,ArctosDB/arctos,https://api.github.com/repos/ArctosDB/arctos,closed,rebuild startup stuff,Priority-Critical (Arctos is broken),"missing, onRequestStart, rolecheck don't alway...",high_priority,0
4,2.125953e+10,IssuesEvent,2022-04-13 01:31:59,ArctosDB/arctos,https://api.github.com/repos/ArctosDB/arctos,closed,Error pre-creating locality using geography sp...,Priority-Critical (Arctos is broken) Function-...,**Describe the bug**\r\nI noticed I now have t...,high_priority,0
7,2.125992e+10,IssuesEvent,2022-04-13 02:15:55,red/red,https://api.github.com/repos/red/red,closed,Hash becomes invalid after `poke`,status.built status.tested type.bug test.writt...,**Describe the bug**\r\n```\r\n>> h: make hash...,high_priority,0
...,...,...,...,...,...,...,...,...,...,...,...
161657,1.472557e+10,IssuesEvent,2021-01-06 05:04:49,breindy/autoCraigslistScraper,https://api.github.com/repos/breindy/autoCraig...,closed,Create Web Scraper Endpoint,Backend Web Scraper,with AlertListings Collections passed in to q...,not_high_priority,1
161658,1.666113e+10,IssuesEvent,2021-06-06 10:45:19,XanderTerbl1/peak-planner,https://api.github.com/repos/XanderTerbl1/peak...,closed,DateTime not updated if passed through custom ...,bug,For some reason the DateTime (in Planner) is n...,not_high_priority,1
161659,1.469324e+10,IssuesEvent,2021-01-03 07:45:57,apache/incubator-doris,https://api.github.com/repos/apache/incubator-...,closed,Some errors in the document need to be rectified,kind/documentation,"Hi, all.\r\nI found some mistakes in our docum...",not_high_priority,1
161660,1.727417e+10,IssuesEvent,2021-07-23 02:15:39,operate-first/alerts,https://api.github.com/repos/operate-first/alerts,closed,CephMonHighNumberOfLeaderChanges,alert:boom: environment/moc/zero resolved,<!-- ID: 0x7b7d2f7b7d2f7b6578706f727465645f6e6...,not_high_priority,1


In [14]:
# Resets the index
all_priority.reset_index(inplace=True)
all_priority

,index,id,type,created_at,repo,repo_url,action,title,labels,body,class,label
0,0,2.125950e+10,IssuesEvent,2022-04-13 01:29:38,federicazoe/gradetools,https://api.github.com/repos/federicazoe/grade...,opened,Comprehensive vignette,high priority,- [ ] Create new vignette on project team grad...,high_priority,0
1,1,2.125952e+10,IssuesEvent,2022-04-13 01:31:43,darrylsyms/fretwise-app,https://api.github.com/repos/darrylsyms/fretwi...,closed,App crash,bug requires testing high priority,"App crashes when Darryl visits ""MessagesScreen...",high_priority,0
2,3,2.125953e+10,IssuesEvent,2022-04-13 01:31:58,ArctosDB/arctos,https://api.github.com/repos/ArctosDB/arctos,closed,rebuild startup stuff,Priority-Critical (Arctos is broken),"missing, onRequestStart, rolecheck don't alway...",high_priority,0
3,4,2.125953e+10,IssuesEvent,2022-04-13 01:31:59,ArctosDB/arctos,https://api.github.com/repos/ArctosDB/arctos,closed,Error pre-creating locality using geography sp...,Priority-Critical (Arctos is broken) Function-...,**Describe the bug**\r\nI noticed I now have t...,high_priority,0
4,7,2.125992e+10,IssuesEvent,2022-04-13 02:15:55,red/red,https://api.github.com/repos/red/red,closed,Hash becomes invalid after `poke`,status.built status.tested type.bug test.writt...,**Describe the bug**\r\n```\r\n>> h: make hash...,high_priority,0
...,...,...,...,...,...,...,...,...,...,...,...,...
125075,161657,1.472557e+10,IssuesEvent,2021-01-06 05:04:49,breindy/autoCraigslistScraper,https://api.github.com/repos/breindy/autoCraig...,closed,Create Web Scraper Endpoint,Backend Web Scraper,with AlertListings Collections passed in to q...,not_high_priority,1
125076,161658,1.666113e+10,IssuesEvent,2021-06-06 10:45:19,XanderTerbl1/peak-planner,https://api.github.com/repos/XanderTerbl1/peak...,closed,DateTime not updated if passed through custom ...,bug,For some reason the DateTime (in Planner) is n...,not_high_priority,1
125077,161659,1.469324e+10,IssuesEvent,2021-01-03 07:45:57,apache/incubator-doris,https://api.github.com/repos/apache/incubator-...,closed,Some errors in the document need to be rectified,kind/documentation,"Hi, all.\r\nI found some mistakes in our docum...",not_high_priority,1
125078,161660,1.727417e+10,IssuesEvent,2021-07-23 02:15:39,operate-first/alerts,https://api.github.com/repos/operate-first/alerts,closed,CephMonHighNumberOfLeaderChanges,alert:boom: environment/moc/zero resolved,<!-- ID: 0x7b7d2f7b7d2f7b6578706f727465645f6e6...,not_high_priority,1


In [15]:
# Does this need to be more balanced?
all_priority["label"].value_counts()

label
1    68859
0    56221
Name: count, dtype: int64

In [16]:
all_priority.drop(columns=["index"] , inplace= True)

/tmp/ipykernel_321717/2269905686.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_priority.drop(columns=["index"] , inplace= True)


In [17]:
# Copy content of body to a new col named text
# Why?
all_priority["text"] = all_priority["body"]
all_priority.tail()

/tmp/ipykernel_321717/1678713137.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_priority["text"] = all_priority["body"]


,id,type,created_at,repo,repo_url,action,title,labels,body,class,label,text
125075,1.472557e+10,IssuesEvent,2021-01-06 05:04:49,breindy/autoCraigslistScraper,https://api.github.com/repos/breindy/autoCraig...,closed,Create Web Scraper Endpoint,Backend Web Scraper,with AlertListings Collections passed in to q...,not_high_priority,1,with AlertListings Collections passed in to q...
125076,1.666113e+10,IssuesEvent,2021-06-06 10:45:19,XanderTerbl1/peak-planner,https://api.github.com/repos/XanderTerbl1/peak...,closed,DateTime not updated if passed through custom ...,bug,For some reason the DateTime (in Planner) is n...,not_high_priority,1,For some reason the DateTime (in Planner) is n...
125077,1.469324e+10,IssuesEvent,2021-01-03 07:45:57,apache/incubator-doris,https://api.github.com/repos/apache/incubator-...,closed,Some errors in the document need to be rectified,kind/documentation,"Hi, all.\r\nI found some mistakes in our docum...",not_high_priority,1,"Hi, all.\r\nI found some mistakes in our docum..."
125078,1.727417e+10,IssuesEvent,2021-07-23 02:15:39,operate-first/alerts,https://api.github.com/repos/operate-first/alerts,closed,CephMonHighNumberOfLeaderChanges,alert:boom: environment/moc/zero resolved,<!-- ID: 0x7b7d2f7b7d2f7b6578706f727465645f6e6...,not_high_priority,1,<!-- ID: 0x7b7d2f7b7d2f7b6578706f727465645f6e6...
125079,1.704859e+10,IssuesEvent,2021-07-06 05:33:06,near/near-indexer-for-explorer,https://api.github.com/repos/near/near-indexer...,closed,Track testnet/mainnet releases and deploy inde...,SRE,"@chefsale I believe you already work on this, ...",not_high_priority,1,"@chefsale I believe you already work on this, ..."


In [18]:
# Make a new dataframe with only text, label and class cols.
all_priority_subset = all_priority[["text" , "label" , "class"]]

In [19]:
all_priority_subset

,text,label,class
0,- [ ] Create new vignette on project team grad...,0,high_priority
1,"App crashes when Darryl visits ""MessagesScreen...",0,high_priority
2,"missing, onRequestStart, rolecheck don't alway...",0,high_priority
3,**Describe the bug**\r\nI noticed I now have t...,0,high_priority
4,**Describe the bug**\r\n```\r\n>> h: make hash...,0,high_priority
...,...,...,...
125075,with AlertListings Collections passed in to q...,1,not_high_priority
125076,For some reason the DateTime (in Planner) is n...,1,not_high_priority
125077,"Hi, all.\r\nI found some mistakes in our docum...",1,not_high_priority
125078,<!-- ID: 0x7b7d2f7b7d2f7b6578706f727465645f6e6...,1,not_high_priority


In [20]:
# What is this for?
# all_priority_subset.to_csv("subset_priority_dataset.csv")

In [21]:
# Convert to string
all_priority_subset["text_str"] = all_priority_subset['text'].astype(str)

/tmp/ipykernel_321717/3106036705.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_priority_subset["text_str"] = all_priority_subset['text'].astype(str)


In [22]:
all_priority_subset

,text,label,class,text_str
0,- [ ] Create new vignette on project team grad...,0,high_priority,- [ ] Create new vignette on project team grad...
1,"App crashes when Darryl visits ""MessagesScreen...",0,high_priority,"App crashes when Darryl visits ""MessagesScreen..."
2,"missing, onRequestStart, rolecheck don't alway...",0,high_priority,"missing, onRequestStart, rolecheck don't alway..."
3,**Describe the bug**\r\nI noticed I now have t...,0,high_priority,**Describe the bug**\r\nI noticed I now have t...
4,**Describe the bug**\r\n```\r\n>> h: make hash...,0,high_priority,**Describe the bug**\r\n```\r\n>> h: make hash...
...,...,...,...,...
125075,with AlertListings Collections passed in to q...,1,not_high_priority,with AlertListings Collections passed in to q...
125076,For some reason the DateTime (in Planner) is n...,1,not_high_priority,For some reason the DateTime (in Planner) is n...
125077,"Hi, all.\r\nI found some mistakes in our docum...",1,not_high_priority,"Hi, all.\r\nI found some mistakes in our docum..."
125078,<!-- ID: 0x7b7d2f7b7d2f7b6578706f727465645f6e6...,1,not_high_priority,<!-- ID: 0x7b7d2f7b7d2f7b6578706f727465645f6e6...


In [23]:
# Clean the data.
all_priority_subset["text_clean"] = all_priority_subset["text_str"].map(preprocess_text)

/tmp/ipykernel_321717/63294665.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_priority_subset["text_clean"] = all_priority_subset["text_str"].map(preprocess_text)


In [24]:
# Make a subset with text_clean and label
priority_label_text = all_priority_subset[["text_clean" , "label"]]
priority_label_text

,text_clean,label
0,create new vignette on project team grading wi...,0
1,app crashes when darryl visits messagesscreen ...,0
2,missing onrequeststart rolecheck dont always w...,0
3,describe the bug i noticed i now have the opti...,0
4,describe the bug h make hash 1 2 3 4 5 make ha...,0
...,...,...
125075,with alertlistings collections passed in to qu...,1
125076,for some reason the datetime in planner is not...,1
125077,hi all i found some mistakes in our document s...,1
125078,id 0x7b7d2f7b7d2f7b6578706f727465645f6e616d657...,1


In [25]:
priority_label_text.dropna(inplace=True)
priority_label_text.reset_index(inplace=True)
priority_label_text.drop(columns=["index"] , inplace= True)

priority_label_text

/tmp/ipykernel_321717/1038009599.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  priority_label_text.dropna(inplace=True)
/tmp/ipykernel_321717/1038009599.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  priority_label_text.drop(columns=["index"] , inplace= True)


,text_clean,label
0,create new vignette on project team grading wi...,0
1,app crashes when darryl visits messagesscreen ...,0
2,missing onrequeststart rolecheck dont always w...,0
3,describe the bug i noticed i now have the opti...,0
4,describe the bug h make hash 1 2 3 4 5 make ha...,0
...,...,...
112304,with alertlistings collections passed in to qu...,1
112305,for some reason the datetime in planner is not...,1
112306,hi all i found some mistakes in our document s...,1
112307,id 0x7b7d2f7b7d2f7b6578706f727465645f6e616d657...,1


In [28]:
# Clean dataset with clean text and labels.
# 0 = high priority, 1 = medium priority, 2 low priority.
priority_label_text.to_csv("csv/priority_high_or_not_high_clean.csv")

In [27]:
df2 = pd.read_csv("priority_high_med_low_dataset_clean.csv", index_col=0)
df2

,text_clean,label
0,create new vignette on project team grading wi...,0
1,app crashes when darryl visits messagesscreen ...,0
2,missing onrequeststart rolecheck dont always w...,0
3,describe the bug i noticed i now have the opti...,0
4,describe the bug h make hash 1 2 3 4 5 make ha...,0
...,...,...
112304,with alertlistings collections passed in to qu...,1
112305,for some reason the datetime in planner is not...,1
112306,hi all i found some mistakes in our document s...,1
112307,id 0x7b7d2f7b7d2f7b6578706f727465645f6e616d657...,1
